In [ ]:
import numpy as np
from PIL import Image, ImageOps
import cv2
import time
import matplotlib.pyplot as plt
def translate_image_without_opencv(image, x, y):
    # Create a blank canvas the same size as the input image
    width, height = image.size
    translated_image = Image.new("L", (width, height))

    # Loop through each pixel in the image and shift it by (x, y)
    for i in range(width):
        for j in range(height):
            new_x, new_y = i + x, j + y
            if 0 <= new_x < width and 0 <= new_y < height:
                translated_image.putpixel((new_x, new_y), image.getpixel((i, j)))

    return translated_image
def translate_image_with_opencv(image, x, y):
    # Get the dimensions of the image
    height, width = image.shape[:2]

    # Create a translation matrix for shifting the image
    translation_matrix = np.float32([[1, 0, x], [0, 1, y]])

    # Apply the translation matrix using OpenCV's warpAffine function
    return cv2.warpAffine(image, translation_matrix, (width, height))
def rotate_image_without_opencv(image, angle):
    # Calculate the rotation matrix manually
    angle_radians = np.radians(angle)
    cos_theta = np.cos(angle_radians)
    sin_theta = np.sin(angle_radians)
    width, height = image.size
    rotated_image = Image.new("L", (width, height))

    # Calculate the center of the image
    center_x, center_y = width // 2, height // 2

    # Loop through each pixel and calculate its new rotated position
    for i in range(width):
        for j in range(height):
            x = i - center_x
            y = j - center_y
            new_x = int(center_x + x * cos_theta - y * sin_theta)
            new_y = int(center_y + x * sin_theta + y * cos_theta)
            if 0 <= new_x < width and 0 <= new_y < height:
                rotated_image.putpixel((new_x, new_y), image.getpixel((i, j)))

    # Return the rotated image
    return rotated_image
def rotate_image_with_opencv(image, angle):
    # Get the dimensions of the image
    height, width = image.shape[:2]

    # Define the center and create a rotation matrix using OpenCV
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, -angle, 1.0)

    # Apply the rotation matrix using warpAffine
    return cv2.warpAffine(image, rotation_matrix, (width, height))
def histogram_equalization_without_opencv(image):
    # Get the histogram of the image and normalize it to get the CDF
    histogram = np.array(image.histogram())
    cdf = np.cumsum(histogram) / histogram.sum()

    # Use the CDF to map the pixel values to their equalized values
    equalized = np.array([int(cdf[pixel] * 255) for pixel in image.getdata()]).reshape(image.size[::-1])

    # Return the equalized image as a PIL Image
    return Image.fromarray(equalized.astype(np.uint8))

def histogram_equalization_with_opencv(image):
    # Apply OpenCV's built-in function for histogram equalization
    return cv2.equalizeHist(image)
def main():
    image_path = "image.jpg"  
    image = ImageOps.grayscale(Image.open(image_path))
    image_np = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Translation offsets
    x_offset, y_offset = 50, 30

    # Perform translation without OpenCV
    translated_image_without = translate_image_without_opencv(image, x_offset, y_offset)
    translated_image_without.save("translated_without_opencv.jpg")

    # Perform translation with OpenCV
    translated_image_with = translate_image_with_opencv(image_np, x_offset, y_offset)
    cv2.imwrite("translated_with_opencv.jpg", translated_image_with)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Translation Without OpenCV")
    plt.imshow(translated_image_without, cmap="gray")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.title("Translation With OpenCV")
    plt.imshow(translated_image_with, cmap="gray")
    plt.axis("off")
    plt.show()

    # Rotation angle
    rotation_angle = 45

    # Perform rotation without OpenCV
    rotated_image_without = rotate_image_without_opencv(image, rotation_angle)
    rotated_image_without.save("rotated_without_opencv.jpg")

    # Perform rotation with OpenCV
    rotated_image_with = rotate_image_with_opencv(image_np, rotation_angle)
    cv2.imwrite("rotated_with_opencv.jpg", rotated_image_with)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Rotation Without OpenCV")
    plt.imshow(rotated_image_without, cmap="gray")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.title("Rotation With OpenCV")
    plt.imshow(rotated_image_with, cmap="gray")
    plt.axis("off")
    plt.show()

    # Perform histogram equalization without OpenCV
    equalized_image_without = histogram_equalization_without_opencv(image)
    equalized_image_without.save("equalized_without_opencv.jpg")

    # Perform histogram equalization with OpenCV
    equalized_image_with = histogram_equalization_with_opencv(image_np)
    cv2.imwrite("equalized_with_opencv.jpg", equalized_image_with)

    # Show histogram equalization results side by side
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Histogram Equalization Without OpenCV")
    plt.imshow(equalized_image_without, cmap="gray")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.title("Histogram Equalization With OpenCV")
    plt.imshow(equalized_image_with, cmap="gray")
    plt.axis("off")
    plt.show()

if __name__ == "__main__":
    main()